In [3]:
import pandas as pd 
import numpy as np
import random
import datetime
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt 
import utilities as utils
import fractionalDiff as fd
from functools import partial
import datatransformation as dtr

In [4]:
########################## Initialization ##########################
today = datetime.date.today()
bottomDate = today
bottomDate = bottomDate.replace(year = bottomDate.year - 2, month=1, day = 1)
lastYear = today.year - 1
lastYearDate = today 
lastYearDate = lastYearDate.replace(year = bottomDate.year - 1, month=1, day = 1)

selectedZones = ['NORD', 'CNOR', 'CSUD', 'CALA', 'SUD', 'SICI', 'SARD']

sqlIpex = f'SELECT * FROM [4E].[pri].[IPEX] where idzona in \'{selectedZones}\' and data>=\'{bottomDate}\''

sqlBands = 'SELECT * FROM [4E].[par].[v_Calendario]'
sqlCct = f'SELECT * FROM [4E].[pri].[IPEX] where Tipo = \'P\' and mercato = \'MGP\' and data>=\'{bottomDate}\''

ipex = dtr.ExecuteDownloadFromDb(sqlCct)
bands = dtr.ExecuteDownloadFromDb(sqlBands)[['data', 'ora', 'POP']]

In [11]:
ipex.Data.min(), ipex.Data.max()

('2020-01-01', '2022-06-17')

In [104]:
def q_k(x, k):
    return np.quantile(x, k)

defQ = partial(q_k, k = 0.99)

########################## Fixes and creates useful columns ##########################

ipex = ipex.loc[ipex['Ora']!=25]
ipex[('year')] = pd.DatetimeIndex(ipex[('Data')]).year
ipex[('quarter')] = pd.DatetimeIndex(ipex[('Data')]).quarter
ipex[('month')] = pd.DatetimeIndex(ipex[('Data')]).month
ipex[('yearMonth')] = ipex[('year')]*100 + ipex[('month')]
ipex[('yearQuarter')] = ipex[('year')]*100 + ipex[('quarter')]

########################## Creates CCT prices by zones ##########################

pun = ipex.loc[ipex['IDZona']=='PUN', ['Data', 'Ora', 'Valore']]
pzPun = pd.merge(pun, ipex, how='right', left_on=['Data','Ora'], \
    right_on=['Data', 'Ora'])
pzPun.rename(columns={'IDZona':'zone', 'Valore_x':'pun', 'Valore_y':'pz'}, inplace = True)#SS (shape storico?)
pzPun['cct'] = pzPun['pun'] - pzPun['pz']
selectedZones = ['NORD', 'CNOR', 'CSUD', 'CALA', 'SUD', 'SICI', 'SARD']
pzPun = pzPun.loc[pzPun['zone'].isin(selectedZones)]

########################## Calculates statistics for distribution of CCT ##########################

quantiles = np.linspace(0.0, 1.0, 10)
nQuantiles = len(quantiles)
functions = {'cct':[np.mean, np.std] + [partial(q_k, k = q) for q in quantiles]}

pzPunStats = pzPun.groupby(['zone', 'yearQuarter', 'yearMonth']).agg(functions)
pzPunStats = pzPunStats.droplevel(axis = 1, level = [0]).reset_index()
pzPunStats.columns = ['zone', 'yearQuarter', 'yearMonth', 'mean', 'std'] + [f'q_{k}' for k in range(0,nQuantiles)]

########################## Merge for POP calculations ##########################

pzPunBands = pd.merge(pzPun, bands, how = 'right', left_on=['Data', 'Ora'], right_on=['data', 'ora'])[['Data', 'Ora', 'POP', 'pun', 'Mercato', 'zone', 'Tipo', 'pz', 'year',
       'quarter', 'month', 'yearMonth', 'yearQuarter', 'cct']]
pzPunBands.dropna(inplace = True)
pzPunBands = pzPunBands[['Mercato', 'zone', 'year', 'yearQuarter', 'yearMonth', 'Data', 'Ora', 'POP', 'pun', 'pz', 'cct']]

monthlyPrices = pzPunBands[['zone', 'POP', 'yearMonth', 'cct']].groupby(['yearMonth', 'POP', 'zone']).agg({'cct':np.mean})
monthlyPrices = monthlyPrices.reset_index()

blPrices = pzPunBands[['zone', 'POP', 'yearMonth', 'cct']].copy()
blPrices.loc[:, 'POP'] = 'BL'
blPrices = blPrices[['zone', 'POP', 'yearMonth', 'cct']].groupby(['yearMonth', 'POP', 'zone']).agg({'cct':np.mean})
blPrices = blPrices.reset_index()
blPrices = blPrices.loc[blPrices['zone'].isin(selectedZones)]

#monthlyPrices.pivot(index = ['yearMonth', 'POP'], columns = ['zone'], values = ['cct'])

monthlyPrices = pd.concat([monthlyPrices, blPrices], axis = 0)
monthlyPrices = monthlyPrices.loc[monthlyPrices['POP'].isin(['P', 'BL'])]
monthlyPrices = monthlyPrices.loc[monthlyPrices['zone'].isin(selectedZones)]

monthlyPrices['asset'] = monthlyPrices['zone'] + '_' + monthlyPrices['POP']

monthlyPricesByZone = monthlyPrices.pivot(index = ['yearMonth'], columns = ['asset'], values = ['cct']).fillna(0)
monthlyPricesByZone = monthlyPricesByZone.droplevel(level = [0], axis = 1).reset_index()
monthlyPricesByZone = monthlyPricesByZone.loc[~monthlyPricesByZone['CALA_BL'].isna()]
monthlyPricesByZone.drop(columns = ['PUN_BL', 'PUN_P'], inplace = True, errors='ignore')
monthlyPricesByZone.rename(columns={'yearMonth':'date'})
monthlyPricesByZone.tail()

asset,yearMonth,CALA_BL,CALA_P,CNOR_BL,CNOR_P,CSUD_BL,CSUD_P,NORD_BL,NORD_P,SARD_BL,SARD_P,SICI_BL,SICI_P,SUD_BL,SUD_P
25,202202.0,5.540760,11.998994,-1.407371,-3.253620,1.332089,2.909308,-1.417220,-3.253620,1.332089,2.909308,6.234748,15.157804,4.575743,10.876952
26,202203.0,12.187591,26.242437,-3.461610,-7.227326,3.882889,7.803830,-3.461610,-7.227326,3.882817,7.803635,12.176824,26.213451,11.291831,24.696023
27,202204.0,8.007343,10.218537,-1.934443,-3.132104,6.183941,7.876832,-3.872519,-4.771144,6.183941,7.876832,7.364761,12.580669,7.978385,10.218537
28,202205.0,3.398872,6.122022,0.900781,-0.935057,2.770677,4.361312,0.900781,-0.935057,2.875583,4.399039,-22.608810,-11.225076,3.334785,5.941412
29,202206.0,3.331208,8.143264,-0.652560,-1.654794,-0.050651,-0.080572,-0.652560,-1.654794,0.781065,2.094684,3.331208,8.143264,2.856325,7.522289
